In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "advcl"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-4-at-2023-03-17-06-00-7f703ce2.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_advcl_export_big_v2_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[0]

{'id': 44215,
 'annotations': [{'id': 8260,
   'completed_by': 1,
   'result': [{'value': {'start': 0,
      'end': 33,
      'text': 'Ent kui jutuks on kaitsepoliitika',
      'labels': ['advcl_phrases']},
     'id': 'FClqxJpFE_',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['bound']},
     'id': 'Vv3gRWqAb-',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-03-09T16:04:50.375401Z',
   'updated_at': '2023-03-09T16:04:50.375435Z',
   'lead_time': 12.355,
   'prediction': {'id': 44215,
    'model_version': 'undefined',
    'created_ago': '0\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 0,
       'end': 33,
       'text': ['Ent', 'kui', 'jutuks', 'on', 'kaitsepoliitika'],
       'labels': ['advcl_p

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [9]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,44215,Train/aja_sloleht20071217_ud211.conllu,"Ent kui jutuks on kaitsepoliitika , siis paist...",0,33,Ent kui jutuks on kaitsepoliitika,bound,", siis paistab isegi see halvustav tiitel Sten...",91.7,91.7,100.0
1,44216,Train/aja_ml200247_osa_4_ud211.conllu,"Siis , kui taheti jõuga haldusterritoriaalset ...",5,58,", kui taheti jõuga haldusterritoriaalset refor...",bound,"Siis . """,100.0,100.0,100.0
2,44217,Train/aja_horisont2000_osa_3_ud211.conllu,"Ning sugugi pole nii , et igal geenil on vaid ...",21,119,", et igal geenil on vaid üks ülesanne skeemi j...",bound,Ning sugugi pole nii .,100.0,100.0,100.0
3,44218,Train/aja_epl20061216_ud211.conllu,Inimestel on küll sõnaõigus põhiseadusega antu...,169,229,kui vaimselt või tervislikult mitte normaalset...,bound,Inimestel on küll sõnaõigus põhiseadusega antu...,96.3,96.3,100.0
4,44219,Train/aja_luup200106_osa_5_ud211.conllu,See on sama palju kui keskmine ameeriklane aas...,18,49,kui keskmine ameeriklane aastas,free,See on sama palju .,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2745,46962,Train/aja_ee200110_osa_7_ud211.conllu,"Nüüd silmitseb ta oma testi , pea viltu , nagu...",40,60,", nagu kunstikriitik",free,"Nüüd silmitseb ta oma testi , pea viltu , ja ü...",73.1,80.8,84.6
2746,46963,Train/tea_toohoive.conllu,"1989. , 1994. a. tõenäosused on mõõdetud kui b...",41,54,kui brutovoog,bound,"1989. , 1994. a. tõenäosused on mõõdetud , ini...",69.0,69.0,89.7
2747,46964,Train/tea_toohoive.conllu,"1989. , 1994. a. tõenäosused on mõõdetud kui b...",186,209,kui ülemineku tõenäosus,bound,"1989. , 1994. a. tõenäosused on mõõdetud kui b...",75.0,75.0,89.3
2748,46965,Dev/ilu_orlau_ud211.conllu,Ka suvisel ajal on tormiseid pilvealuseid öid ...,83,112,nagu kurjast vaimust vaevatud,free,Ka suvisel ajal on tormiseid pilvealuseid öid ...,93.3,96.7,96.7


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
4,44219,Train/aja_luup200106_osa_5_ud211.conllu,See on sama palju kui keskmine ameeriklane aas...,18,49,kui keskmine ameeriklane aastas,free,See on sama palju .,100.0,100.0,100.0
5,44220,Train/aja_ee200110_osa_5_ud211.conllu,""" Teatud mõttes oleme selleks valmis , sest me...",37,74,", sest meil ei ole ühtegi oma apteeki",free,""" Teatud mõttes oleme selleks valmis , "" ütleb...",90.9,100.0,90.9
6,44221,Train/aja_ml200247_osa_5_ud211.conllu,Värske kapsas maksab Keskturul 4.50 ning porga...,97,116,kui möödunud aastal,free,Värske kapsas maksab Keskturul 4.50 ning porga...,84.2,84.2,89.5
8,44223,Train/tea_eesti_arst_2004_osa_3_ud211.conllu,Arstiabiga kindlustatus teises liikmesriigis e...,24,51,teises liikmesriigis elades,free,Arstiabiga kindlustatus,100.0,100.0,100.0
9,44224,Train/aja_luup200106_osa_4_ud211.conllu,"Uusi asju tuleb nõnda rohkesti peale , et enam...",37,78,", et enam pole lootustki nendega tegeleda",free,Uusi asju tuleb nõnda rohkesti peale .,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2741,46958,Dev/ilu_ruben_ud211.conllu,"Ukse peal võttis mind vastu majandusdirektor ,...",45,75,", hõlmad laiali nagu liputajal",free,Ukse peal võttis mind vastu majandusdirektor .,100.0,100.0,100.0
2742,46959,Dev/ilu_ruben_ud211.conllu,"Ukse peal võttis mind vastu majandusdirektor ,...",61,75,nagu liputajal,free,"Ukse peal võttis mind vastu majandusdirektor ,...",70.0,70.0,80.0
2745,46962,Train/aja_ee200110_osa_7_ud211.conllu,"Nüüd silmitseb ta oma testi , pea viltu , nagu...",40,60,", nagu kunstikriitik",free,"Nüüd silmitseb ta oma testi , pea viltu , ja ü...",73.1,80.8,84.6
2748,46965,Dev/ilu_orlau_ud211.conllu,Ka suvisel ajal on tormiseid pilvealuseid öid ...,83,112,nagu kurjast vaimust vaevatud,free,Ka suvisel ajal on tormiseid pilvealuseid öid ...,93.3,96.7,96.7


In [11]:
df_free.to_csv(f"deprel_free_analysis/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
4,44219,Train/aja_luup200106_osa_5_ud211.conllu,See on sama palju kui keskmine ameeriklane aas...,18,49,kui keskmine ameeriklane aastas,free,See on sama palju .,100.0,100.0,100.0
8,44223,Train/tea_eesti_arst_2004_osa_3_ud211.conllu,Arstiabiga kindlustatus teises liikmesriigis e...,24,51,teises liikmesriigis elades,free,Arstiabiga kindlustatus,100.0,100.0,100.0
9,44224,Train/aja_luup200106_osa_4_ud211.conllu,"Uusi asju tuleb nõnda rohkesti peale , et enam...",37,78,", et enam pole lootustki nendega tegeleda",free,Uusi asju tuleb nõnda rohkesti peale .,100.0,100.0,100.0
11,44226,Train/tea_geofyysika.conllu,Tõenäoliselt oli algmaterjaliks oleva isoleeri...,81,103,kui kaks Päikese massi,free,Tõenäoliselt oli algmaterjaliks oleva isoleeri...,100.0,100.0,100.0
14,44229,Train/ilu_valton_ud211.conllu,"Kui ta tiiva alla puges , imestas , et teised ...",0,25,"Kui ta tiiva alla puges ,",free,"imestas , et teised on juba ees .",100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2723,46940,Train/aja_horisont2000_osa_5_ud211.conllu,Ent kuidas jääda ellu ekvaatori põletava päike...,53,107,", kui nahas ega karvades pole teragi kaitsvat ...",free,Ent kuidas jääda ellu ekvaatori põletava päike...,100.0,100.0,100.0
2724,46941,Train/aja_ee199920_osa_2_ud211.conllu,"Rootsi on liiga väike , et omada nelja iseseis...",22,55,", et omada nelja iseseisvat panka",free,Rootsi on liiga väike .,100.0,100.0,100.0
2726,46943,Train/aja_pm19980926e_ud211.conllu,"Selle juurde võib tagasi pöörduda hiljem , kui...",41,95,", kui on lisandunud pilte järgnevatest eluoluk...",free,Selle juurde võib tagasi pöörduda hiljem .,100.0,100.0,100.0
2733,46950,Train/aja_ee199920_osa_3_ud211.conllu,Kommertstelevisioon on USAs ja Suurbritannias ...,78,158,", samas kui Rootsis on kommertstelevisioon toi...",free,Kommertstelevisioon on USAs ja Suurbritannias ...,100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/{deprel}_free_conserved.csv", sep=";", index=False)

In [10]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,44215,Train/aja_sloleht20071217_ud211.conllu,"Ent kui jutuks on kaitsepoliitika , siis paist...",0,33,Ent kui jutuks on kaitsepoliitika,bound,", siis paistab isegi see halvustav tiitel Sten...",91.7,91.7,100.0
1,44216,Train/aja_ml200247_osa_4_ud211.conllu,"Siis , kui taheti jõuga haldusterritoriaalset ...",5,58,", kui taheti jõuga haldusterritoriaalset refor...",bound,"Siis . """,100.0,100.0,100.0
2,44217,Train/aja_horisont2000_osa_3_ud211.conllu,"Ning sugugi pole nii , et igal geenil on vaid ...",21,119,", et igal geenil on vaid üks ülesanne skeemi j...",bound,Ning sugugi pole nii .,100.0,100.0,100.0
3,44218,Train/aja_epl20061216_ud211.conllu,Inimestel on küll sõnaõigus põhiseadusega antu...,169,229,kui vaimselt või tervislikult mitte normaalset...,bound,Inimestel on küll sõnaõigus põhiseadusega antu...,96.3,96.3,100.0
7,44222,Train/aja_pm19980926e_ud211.conllu,Turvalisus sünnib prognoositavusest : tuttav o...,60,80,distantsi suurenedes,bound,Turvalisus sünnib prognoositavusest : tuttav o...,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2738,46955,Train/aja_ee199920_osa_2_ud211.conllu,"Kui mina oleksin mingist muust rahvusest , sii...",0,40,Kui mina oleksin mingist muust rahvusest,bound,", siis ma küll õpetaksin kodus oma lapsele rah...",100.0,100.0,100.0
2739,46956,Dev/aja_luup200009_ud211.conllu,"Plaadile mahub 4,7 gigabaiti ( GB ) andmeid eh...",118,137,kui VHS-kassettidel,bound,"Plaadile mahub 4,7 gigabaiti ( GB ) andmeid eh...",76.2,81.0,81.0
2743,46960,Train/aja_pm20001004_osa_2_ud211.conllu,Sageli võtab erafirma enda kanda osa riigi üle...,70,95,", kus riik hakkama ei saa",bound,Sageli võtab erafirma enda kanda osa riigi üle...,75.0,83.3,83.3
2746,46963,Train/tea_toohoive.conllu,"1989. , 1994. a. tõenäosused on mõõdetud kui b...",41,54,kui brutovoog,bound,"1989. , 1994. a. tõenäosused on mõõdetud , ini...",69.0,69.0,89.7


In [11]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [12]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
54,44269,Train/aja_luup200106_osa_3_ud211.conllu,"MS : Ma usun , et see aeg , kui õppejõud jooks...",26,69,", kui õppejõud jooksis ringi , et raha teha",redundant comma,"MS : Ma usun , et see aeg , on möödas .",91.7,100.0,91.7
80,44295,Train/aja_pm20001004_osa_3_ud211.conllu,Kuigi ametlikult Tallinna Keskturu inseneri ja...,0,161,Kuigi ametlikult Tallinna Keskturu inseneri ja...,redundant comma,", õnnestus keskkriminaalpolitseil tänu salajas...",82.6,82.6,95.7
115,44330,Train/tea_toohoive.conllu,"Kui töötaolek on madal , on streigifond suur j...",0,22,Kui töötaolek on madal,redundant comma,", on streigifond suur ja ametiühingud võivad t...",100.0,100.0,100.0
119,44334,Train/aja_luup200106_osa_3_ud211.conllu,Kavandamaks riiklikke meetmeid olukorra parand...,0,53,Kavandamaks riiklikke meetmeid olukorra parand...,redundant comma,", peame teadma , millise juure all ikkagi peit...",100.0,100.0,100.0
158,44373,Train/tea_toohoive.conllu,"Selle protsessi käigus on Eestis , võrreldes t...",33,70,", võrreldes teiste üleminekuriikidega",redundant comma,"Selle protsessi käigus on Eestis , läinud kadu...",90.0,96.7,93.3
...,...,...,...,...,...,...,...,...,...,...,...
2664,46881,Train/ilu_kivirahk_ud211.conllu,"Kuid kuna päev üha kestis , ja nemad olid andn...",0,64,"Kuid kuna päev üha kestis , ja nemad olid andn...",redundant comma,", tuli edasi liikuda .",100.0,100.0,100.0
2680,46897,Train/tea_geofyysika.conllu,Geoloogia keskendub maakoore ehituse ja arengu...,63,143,eriti pöörates tähelepanu mineraalidele ja min...,redundant comma,Geoloogia keskendub maakoore ehituse ja arengu...,100.0,100.0,100.0
2720,46937,Train/aja_ml200247_osa_5_ud211.conllu,"Vähe sellest , raha hakkab hoopis minema , ses...",0,12,Vähe sellest,redundant comma,", raha hakkab hoopis minema , sest kasutusvald...",88.2,88.2,100.0
2740,46957,Train/aja_sloleht20071217_ud211.conllu,"Kuigi kümnest vormi selga ajanud mehest neli ,...",0,106,"Kuigi kümnest vormi selga ajanud mehest neli ,...",redundant comma,", löödi laupäeval Kalevi spordihallis suurepär...",None,None,None


In [13]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [14]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
13,44228,Train/ilu_maailm_ud211.conllu,Viimastel päevadel peaaegu ununenud hambavalu ...,58,84,; kui ühiselamusse jõudsin,unnatural,Viimastel päevadel peaaegu ununenud hambavalu ...,88.2,94.1,94.1
27,44242,Dev/aja_ee199920_osa4_ud211.conllu,"Lihtsalt öeldes on etnofuturism väide , et kõi...",0,15,Lihtsalt öeldes,unnatural,"on etnofuturism väide , et kõigi soome-ugri ra...",100.0,100.0,100.0
30,44245,Train/aja_sloleht20071217_ud211.conllu,Majandusministri arvates on just helde jootrah...,0,24,Majandusministri arvates,unnatural,"on just helde jootraha jagamine üks põhjus , m...",76.5,88.2,88.2
41,44256,Train/tea_AA_05_6_ud211.conllu,Et paremini esile tuua infotöötluse korralduse...,0,110,Et paremini esile tuua infotöötluse korralduse...,unnatural,"on joonisel 1 näidatud , kuidas lähtekoodist (...",94.7,98.2,96.5
43,44258,Train/aja_luup200202_osa_2_ud211.conllu,Kuigi Martin Lutheri nimega seotud kiriklikust...,0,112,Kuigi Martin Lutheri nimega seotud kiriklikust...,unnatural,on see teema endiselt hell ning ka 21. sajandi...,95.2,95.2,95.2
...,...,...,...,...,...,...,...,...,...,...,...
2625,46842,Train/ilu_vilep_ud211.conllu,Majanumbreid loendades jõudis ta lõpuks kahekü...,0,22,Majanumbreid loendades,unnatural,jõudis ta lõpuks kahekümne neljandani ja astus...,92.3,100.0,92.3
2665,46882,Train/aja_ee199920_osa_1_ud211.conllu,Kui suusakeskus enam arveid maksta ei jõudnud ...,0,47,"Kui suusakeskus enam arveid maksta ei jõudnud ,",unnatural,vormistati võlg ümber 16protsendiliseks laenuk...,90.0,90.0,90.0
2683,46900,Dev/aja_ml200247_osa6_ud211.conllu,"Saates lühinumbrile 1311 SMSi sisuga "" INFO TA...",0,54,"Saates lühinumbrile 1311 SMSi sisuga "" INFO TA...",unnatural,saabub hetke pärast vastus Taisto liinide Tart...,62.5,62.5,87.5
2686,46903,Train/aja_pm19980926e_ud211.conllu,Küsimusi esitades aitab ta anda värvile rohkem...,0,17,Küsimusi esitades,unnatural,aitab ta anda värvile rohkem omadusi : kas sa ...,90.5,90.5,100.0


In [15]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [16]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la


In [17]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [18]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
75,44290,Dev/aja_ee199920_osa4_ud211.conllu,Vahel -- kohates mõnd tuntud mõtlejat -- olen ...,6,37,-- kohates mõnd tuntud mõtlejat,other redundant punctuation,Vahel -- olen küsinud : kuidas mõjutab eesti ü...,92.9,100.0,92.9
455,44670,Dev/ilu_ruben_ud211.conllu,Mõtlesin - endiselt koondamisotsust oodates - ...,11,43,endiselt koondamisotsust oodates,other redundant punctuation,Mõtlesin - - et lähen alla baari ja teen seal ...,87.5,93.8,93.8
535,44750,Train/tea_AA_05_6_ud211.conllu,"Siiski ei tohiks eelnevat mõista selliselt , e...",158,232,võimaldada hakkama saada tohutute andmehulkade...,other redundant punctuation,"Siiski ei tohiks eelnevat mõista selliselt , e...",58.3,62.5,87.5
689,44904,Train/aja_ee200110_osa_8_ud211.conllu,Mis iganes Euroopa linna tema shokk-rock-karav...,125,148,", käes "" Peatage Saatan",other redundant punctuation,Mis iganes Euroopa linna tema shokk-rock-karav...,76.5,82.4,88.2
748,44963,Train/aja_ee200110_osa_5_ud211.conllu,Tulles tagasi inimese juurde - kas me olemegi ...,0,28,Tulles tagasi inimese juurde,other redundant punctuation,- kas me olemegi nii ettearvamatud ?,100.0,100.0,100.0
1403,45619,Train/ilu_kanep_ud211.conllu,Ma ise põhimõtteliselt hea meelega väldiksin s...,136,270,", kuna niikuinii nad ei mõista miskit mõistlik...",other redundant punctuation,Ma ise põhimõtteliselt hea meelega väldiksin s...,87.0,87.0,100.0
1522,45738,Train/aja_ee199920_osa_3_ud211.conllu,"McNeal on välja mõelnud sõna "" filiarhaat "" --...",44,79,-- nagu patriarhaat või matriarhaat,other redundant punctuation,"McNeal on välja mõelnud sõna "" filiarhaat "" --...",81.2,81.2,93.8
1523,45739,Train/aja_ee199920_osa_3_ud211.conllu,"McNeal on välja mõelnud sõna "" filiarhaat "" --...",85,135,kirjeldamaks laste suurenevat mõjuvõimu pereko...,other redundant punctuation,"McNeal on välja mõelnud sõna "" filiarhaat "" --...",93.8,93.8,100.0
1856,46072,Dev/ilu_orlau_ud211.conllu,"Tundes nii tolle hirmu sügavust ja tagamaid , ...",0,139,"Tundes nii tolle hirmu sügavust ja tagamaid , ...",other redundant punctuation,— sain Juudasest aru .,100.0,100.0,100.0
1934,46150,Train/aja_epl20061216_ud211.conllu,""" On oluline , et kandidaatriigid täidaksid om...",87,178,", lisades samas : "" Ja oluline on , et EL jääk...",other redundant punctuation,""" On oluline , et kandidaatriigid täidaksid om...",66.7,66.7,86.7


In [19]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:17:15.792262: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:17:16.312319: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:17:18.099134: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:17:18.099292: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data



1227it [04:48,  4.25it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")